In [1]:
! pip install BeautifulSoup4

In [1]:
from bs4 import BeautifulSoup
from csv import writer
import requests
import time
import random
url="https://catala.habitaclia.com/habitatges-terrassa.htm"
url_composta1 = "https://catala.habitaclia.com/habitatges-terrassa-"
url_composta2 = 1
url_composta3 = ".htm"
str_https = "https:"


In [2]:
#Obtenim el codi HTML de l'enllaç url
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

In [3]:
#A partir del codi html extraiem tots els "tags" de tipus "article"
articles = soup.find_all("article")
file = open('dataset_houses_1.csv','w')
file.write("Preu,Zona,Superficie,Habitacions,Lavabos,Preu metre,Calefaccio,Aire acondicionat,Any construccio,Aparcament,Piscina,Ascensor,Planta,Consum,Venda/lloguer\n")
file.close()

In [4]:
#Obtenim el codi HTML de l'enllaç url
page = requests.get(url)


while page.status_code == 200: #Mentre seguim trobant pàgines de cerca anem iterant
    try:
        soup = BeautifulSoup(page.content, "html.parser")
        print("ESCRAPEJANT PÀGINA ", url_composta2)
        #A partir del codi html extraiem tots els "tags" de tipus "article"
        articles = soup.find_all("article")
        if len(articles) <= 2:
            break
        #Per cada article de la llista d'articles cerquem l'atribut "data-href" (és on hi ha tots els detalls de l'anunci)
        for article in articles:
            try:
                if article is None:
                    continue
                data_href = article.get("data-href")
                if data_href is None or "terrassa" not in data_href:
                    continue

                data_href = str_https + data_href
                if(data_href is not None):
                    time.sleep(random.uniform(1.5, 7.5))
                    page_anunci = requests.get(data_href)
                    anunci = BeautifulSoup(page_anunci.content, "html.parser")
                    an_preu = anunci.find("span", {"itemprop": "price"}).getText()
                    if an_preu is not None:
                        an_preu = an_preu[0:len(an_preu)-2]
                    else:
                        continue #No ens interesa obtenir cap habitatge sense una barri especificat
                    an_barri = anunci.find("a", {"class": "jqVerMapaZonaTooltip"})
                    if(an_barri):
                        an_barri = an_barri.getText().strip()
                    else:
                        continue #No ens interesa obtenir cap habitatge sense una barri especificat
                    features = anunci.find_all("li", {"class": "feature"})
                    an_metres = "_"
                    an_hab = "_"
                    an_banys = "_"
                    an_preu_metre = "_"
                    an_calefaccio = "No"
                    an_aireAcondicionat = "No"
                    an_any = "_"
                    an_aparcament = "No"
                    an_piscina = "No"
                    an_ascensor = "No"
                    an_numPlanta = "_"
                    an_consum = "_"

                    if(len(features) == 8):
                        an_metres = features[0].find("strong").getText()
                        an_hab = features[1].find("strong").getText()
                        an_banys = features[2].find("strong").getText()
                        an_preu_metre = features[3].find("strong").getText()


                    art_caracteristiques = anunci.find_all("article", {"class": "has-aside"})
                    for art in art_caracteristiques:
                        try:
                            caracteristiques = art.find("h3", string = "Característiques generals")
                            if (caracteristiques):
                                lis = art.find_all("li")
                                for li in lis:
                                    if("Calefaccio" in li.getText().strip()):
                                        an_calefaccio = "Si"
                                    elif("Aire Acondicionat" in li.getText().strip()):
                                        an_aireAcondicionat = "Si"
                                    elif("Plaça aparcament" in li.getText().strip()):
                                        an_aparcament = "Si"
                                    elif("Piscina" in li.getText().strip()):
                                        an_piscina = "Si"
                                    elif("Planta numero" in li.getText().strip()):
                                        an_numPlanta = li.getText().strip().split()[2]
                                    elif("Any construccio" in li.getText().strip()):
                                        an_any = li.getText().strip().split()[2]
                                    else:
                                        if(li.find("div", {"class": "rating c-A"}) is not None):
                                            an_consum = "A"
                                        elif(li.find("div", {"class": "rating c-B"}) is not None):
                                            an_consum = "B"
                                        elif(li.find("div", {"class": "rating c-C"}) is not None):
                                            an_consum = "C"
                                        elif(li.find("div", {"class": "rating c-D"}) is not None):
                                            an_consum = "D"
                                        elif(li.find("div", {"class": "rating c-E"}) is not None):
                                            an_consum = "E"
                                        elif(li.find("div", {"class": "rating c-F"}) is not None):
                                            an_consum = "F"
                                        elif(li.find("div", {"class": "rating c-G"}) is not None):
                                            an_consum = "G"

                            caracteristiques = art.find("h3", string = "Equipament comunitari")
                            if (caracteristiques):
                                lis = art.find_all("li")
                                for li in lis:
                                    if("Ascensor" in li.getText().strip()):
                                        an_ascensor = "Si"

                        except:
                            break
                      

                    an_preu = "" + an_preu.replace(".","")
                    an_preu_metre = "" + an_preu_metre.replace(".","")

                    print("Preu:",an_preu)
                    print("Barri:",an_barri)
                    print("Metres:",an_metres)
                    print("Habitacions:",an_hab)
                    print("Banys:",an_banys)
                    print("Preu metre:",an_preu_metre)
                    print("Calefacció:",an_calefaccio)
                    print("Aire acondicionat:",an_aireAcondicionat)
                    print("Any:",an_any)
                    print("Aparcament:",an_aparcament)
                    print("Piscina:",an_piscina)
                    print("Ascensor:",an_ascensor)
                    print("Planta:",an_numPlanta)
                    print("Consum:",an_consum)

                    file = open('dataset_houses_1.csv','a')
                    str_toWrite = an_preu + "," + an_barri + "," + an_metres + "," + an_hab + "," + an_banys + "," + an_preu_metre + "," + an_calefaccio + "," + an_aireAcondicionat + "," + an_any + "," + an_aparcament + "," + an_piscina + "," + an_ascensor + "," + an_numPlanta + "," + an_consum + ",Venta\n"
                    file.write(str_toWrite)
                    file.close()
            except:
                continue
        page = requests.get(url_composta1 + str(url_composta2) + url_composta3)
        url_composta2 = url_composta2 + 1
    except:
        continue
print("Fi del scrapping")

ESCRAPEJANT PÀGINA  1
Preu: 189000
Barri: Les Arenes-La Grípia
Metres: 112
Habitacions: 4
Banys: 2
Preu metre: 1688
Calefacció: Sí
Aire acondicionat: Sí
Any: 1992
Aparcament: No
Piscina: No
Ascensor: Sí
Planta: _
Consum: E
Preu: 198500
Barri: Cementiri Vell
Metres: 100
Habitacions: 4
Banys: 2
Preu metre: 1985
Calefacció: Sí
Aire acondicionat: No
Any: 1979
Aparcament: No
Piscina: No
Ascensor: Sí
Planta: _
Consum: G
Preu: 225000
Barri: Ca n´Aurell
Metres: 94
Habitacions: 3
Banys: 1
Preu metre: 2394
Calefacció: Sí
Aire acondicionat: Sí
Any: 1961
Aparcament: No
Piscina: No
Ascensor: Sí
Planta: _
Consum: G
Preu: 248000
Barri: Sant Pere
Metres: 82
Habitacions: 2
Banys: 2
Preu metre: 3024
Calefacció: Sí
Aire acondicionat: Sí
Any: 2004
Aparcament: No
Piscina: No
Ascensor: Sí
Planta: _
Consum: G
Preu: 180000
Barri: Ca n´Anglada
Metres: 108
Habitacions: 3
Banys: 1
Preu metre: 1667
Calefacció: Sí
Aire acondicionat: Sí
Any: 1994
Aparcament: No
Piscina: No
Ascensor: No
Planta: _
Consum: G
Preu: 179